In [1]:
# generate token for every ids.

import os
import pandas as pd
from transformers import BertTokenizer
from utils.seqlab import Index_Dictionary
from utils.utils import merge_kmer

path = os.path.join("prediction", "log", "prediction_log.csv")
df = pd.read_csv(path)

tokenizer = BertTokenizer.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))
df["input_tokens"] = df.apply(lambda x: " ".join(tokenizer.convert_ids_to_tokens([int(a) for a in x["input_ids"].split(" ")])), axis=1)
df["prediction_tokens"] = df.apply(lambda x: " ".join([Index_Dictionary[a] for a in [int(b) for b in x["prediction_ids"].split(" ")]]), axis=1)
df["target_tokens"] = df.apply(lambda x: " ".join([Index_Dictionary[a] for a in [int(b) for b in x["target_ids"].split(" ")]]), axis=1)
df["sequence"] = df.apply(lambda x: merge_kmer(x["input_tokens"].split(" ")), axis=1)
df["prediction"] = df.apply(lambda x: merge_kmer(x["prediction_tokens"].split(" ")), axis=1)
df["target"] = df.apply(lambda x: merge_kmer(x["target_tokens"].split(" ")), axis=1)
df["avg_f1_score"] = df.apply(lambda x: (x["f1_score-EEE"] + x["f1_score-EEi"] + x["f1_score-Eii"] + x["f1_score-iEE"] + x["f1_score-iiE"] + x["f1_score-iii"])/6, axis=1)
df.to_csv(os.path.join("prediction", "log", "prediction_log_complete.csv"), index=False)

c:\.virtualenv\deep-learning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sorted = df.sort_values(by=["avg_f1_score"], ascending=False)
sorted[sorted["avg_f1_score"] == 1].shape

(45, 34)

In [21]:
import pandas as pd

df = pd.read_csv(os.path.join("prediction", "log", "prediction_log_complete.csv"))
ss_labels = ["iiE", "iEE", "EEi", "Eii"]

all_correct_df = df[df["avg_f1_score"] == 1]
print(all_correct_df.shape)
tokens, predictions, targets = [], [], []
for i, r in all_correct_df.iterrows():
    arr_tokens = r["input_tokens"].split(" ")
    arr_predictions = r["prediction_tokens"].split(" ")
    arr_targets = r["target_tokens"].split(" ")

    all_clear = all([a == b for a, b in zip(arr_predictions, arr_targets)])
    if not all_clear:
        raise ValueError(f"{arr_predictions}\n{arr_targets}")

    for i, j, k in zip(arr_tokens, arr_predictions, arr_targets):
        if k in ss_labels:
            tokens.append(i)
            predictions.append(j)
            targets.append(k)

ndf = pd.DataFrame(data={
    "token": tokens,
    "prediction": predictions,
    "target": targets
})
print(ndf.shape)
correct_df = ndf[ndf["prediction"] == ndf["target"]]
print(correct_df.shape)
false_df = ndf[ndf["prediction"] != ndf["target"]]
print(false_df.shape)

(115, 34)
(320, 3)
(320, 3)
(0, 3)
